In [1]:
import pandas as pd

In [2]:
bbaw_eg = pd.read_csv('Data/bbaw_egyptian.csv')
thesaurus_earlier = pd.read_csv('Data/thesaurus_earlier.csv')
thesaurus_late = pd.read_csv('Data/thesaurus_late.csv')

# BBAW

In [3]:
bbaw_eg.isna().sum()

transcription        7
translation         21
hieroglyphs      65233
dtype: int64

Extract only how have hieroglyphics "we might use the unlabled again"

In [7]:
# select rows where hieroglyphs is not null
bbaw_eg_w_hieroglyphs = bbaw_eg[bbaw_eg['hieroglyphs'].notna()]

# select rows where hieroglyphs is null
bbaw_eg_wo_hieroglyphs = bbaw_eg[bbaw_eg['hieroglyphs'].isna()]

Cleaning

In [ ]:
# show images in notebook
from IPython.display import Image



In [8]:
bbaw_eg_w_hieroglyphs

,transcription,translation,hieroglyphs
4062,"jr,j-pꜥ,t ḥꜣ,tj-ꜥ ḫtm,w-bj,tj smr-wꜥ,tj jm,j-r...","Hereditary noble and prince, royal seal-bearer...",D21 :Q3 :D36 F4 :D36 L2 -X1 :S19 S29 -U23 -T21...
4063,"j ꜥnḫ.w[.pl] [tp,j].pl-tꜣ swꜣ.t(j) =sn ḥr jz p...","O living ones, who are upon the earth, who sha...",M17 -[? *A26 *?] S34 -/ :Aa1 -G43 -A1 -[? *Z3 ...
4064,"jnk mḥ-jb-n(,j)-nswt m ḥw,t-nṯr rʾ-Nḫn m pr St...","I was a trusted one of the king in the temple,...",W24 :V31 V22 :F34 :N35 -M23 -X1 :N35 G17 R8 -O...
4065,"ꜥq jb.pl ḥr-sꜣ mr,yt jm,j-rʾ-ꜥḥꜥ,w-wr-m-pr-nsw...",A trusted one (lit. one who enters hearts) upo...,G35 F34 -F34 -F34 D2 *Z1 :Aa17 U6 :D21 -M17 -M...
4066,"jw jri̯.n =(j) jz m ḥz(w),t nswt sjqr.n wj ḥm...","I built a tomb through the favour of the king,...",M17 -G43 D4 :N35 M17 -M40 -O34 :O1 :Z1 G17 V2...
...,...,...,...
100731,"wr⸢⸮.pl?⸣ n pḥ,tj =k",Die Großen [... werden fallen? ...] aufgrund d...,G36 -D21 -[? -A21 -?] [? -N35 -?] [? -F4 -?]...
100732,"hmhm =k mj hh nsr.t m-sꜣ ḫꜣs,t nb.t",Dein Kriegsschrei folgt wie der Gluthauch des ...,O4 -G17 -O4 -G17 -A2 -Z2 V31 W19 -M17 O4 -O4 -...
100733,"ḫꜣs,t nb.t twt.w m jb wꜥ.tj fkw =sn tꜣ =sn m-mn,t",... jedes Fremdland ist einträchtig eines einz...,N25 -[? -X1 -?] -Z1 V30 -X1 X1 -G43 -X1 -G43 -...
100734,r zj.tw ṯꜣw r fnḏ =sn jn kꜣ =k,... damit durch deinen Ka Atem für ihre Nasen ...,[? -D21 -?] [? -O35 -?] -Z4A -D54 -X1 -G43 P5 ...
